In [1]:
# Import các thư viện cần thiết
import numpy as np
import cv2
import pandas as pd
import os
import pickle
from google.colab import drive
from google.colab.patches import cv2_imshow

In [22]:
# Connect Google Drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/opencv/'
# Hiển thị tất cả thư mục hoặc file
os.listdir(path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['models', 'face.jpg', 'faces.jpg', 'myface.jpg', 'FaceImage']

In [23]:
faceDetectionModel = path + "/models/res10_300x300_ssd_iter_140000_fp16.caffemodel"

# Mô tả kiến trúc
faceDetectionProto = path + "/models/deploy.prototxt.txt"

# Mô hình mô tả khuôn mặt
faceDescriptor = path + "/models/openface.nn4.small2.v1.t7"

# Sử dụng OpenCV DNN đọc mô hìn nhận diện khuôn mặt
dectectorModel = cv2.dnn.readNetFromCaffe(faceDetectionProto, faceDetectionModel)

# Đọc mô hình mô tả khuôn mặt từ file Torch
descriptorModel = cv2.dnn.readNetFromTorch(faceDescriptor)


In [3]:
# Tạo ra một functions để áp dụng cho nhiều ảnh
def myDetect(image_path):
  # Trích xuất đặc trưng cho một ảnh
  img1 = cv2.imread(image_path)

  # Copy cái ảnh
  image = img1.copy()

  # Lấy chiều cao và chiều rộng của ảnh
  h, w = image.shape[:2]

  # Chuẩn bị dữ liệu đầu vào cho mô hình nhận diện khuôn mặt
  imgBlob = cv2.dnn.blobFromImage(image, 1, (300,300), (104, 177, 123), swapRB=False, crop=False)

  # Thiết lập đầu vào cho mô hình
  dectectorModel.setInput(imgBlob)

  # Thực hiện việc nhận diện khuôn mặt
  detections = dectectorModel.forward()

  # Kiểm tra xem có khuôn mặt nào hay không?
  if (len(detections)>0):
    # Chọn khuôn mặt có độ tin cậy cao nhất (confidence) cao nhất
    i = np.argmax(detections[0, 0, :, 2]) # Giá trị thứ 2 trong 7 giá trị cuối cùng => confidence
    confidence = detections[0, 0, i, 2]

    # Kiểm tra độ tin cậy có lớn hơn 0.5
    if (confidence>0.5):
        # Tính toán hộp bao quanh khuôn mặt
        box = detections[0, 0, i, 3:7]* np.array([w, h, w, h])
        (startX, startY, endX, endY) =box.astype('int')

        # Trích xuất vùng ảnh chứa khuôn mặt ra
        roi = image[startY:endY, startX:endX]
        # cv2_imshow(roi)

        # Chuẩn bị dữ liệu đầu vào cho mô hình trích xuất đặc trưng
        faceBlob = cv2.dnn.blobFromImage(roi, 1/255, (96,96), (0,0,0), swapRB=True, crop=True)

        # Thiết lập đầu vào cho mô hình
        descriptorModel.setInput(faceBlob)

        # Thực hiện việc trích xuất đặc trưng
        vectors = descriptorModel.forward()

        # Print vectors
        return vectors

In [20]:
MyPhotos = '/content/drive/MyDrive/opencv/FaceImage/162001391_TranTien'
os.listdir(MyPhotos)

['imageAAA_40.jpg',
 'imageAAA_20.jpg',
 'imageAAA_60.jpg',
 'imageAAA_80.jpg',
 'imageAAA_100.jpg',
 'imageAAA_120.jpg',
 'imageAAA_160.jpg',
 'imageAAA_140.jpg',
 'imageAAA_180.jpg',
 'imageAAA_220.jpg',
 'imageAAA_200.jpg',
 'imageAAA_240.jpg',
 'imageAAA_280.jpg',
 'imageAAA_260.jpg',
 'imageAAA_300.jpg',
 'imageAAA_340.jpg',
 'imageAAA_320.jpg',
 'imageAAA_360.jpg',
 'imageAAA_400.jpg',
 'imageAAA_380.jpg',
 'imageAAA_420.jpg',
 'imageAAA_440.jpg',
 'imageAAA_460.jpg',
 'imageAAA_480.jpg',
 'imageAAA_500.jpg',
 'imageAAA_520.jpg',
 'imageAAA_540.jpg',
 'imageAAA_560.jpg',
 'imageAAA_580.jpg',
 'imageAAA_600.jpg',
 'imageAAA_640.jpg',
 'imageAAA_620.jpg',
 'imageAAA_660.jpg',
 'imageAAA_680.jpg',
 'imageAAA_700.jpg',
 'imageAAA_720.jpg',
 'imageAAA_760.jpg',
 'imageAAA_740.jpg',
 'imageAAA_780.jpg',
 'imageAAA_800.jpg',
 'imageAAA_820.jpg',
 'imageAAA_840.jpg',
 'imageAAA_880.jpg',
 'imageAAA_860.jpg',
 'imageAAA_900.jpg',
 'imageAAA_920.jpg',
 'imageAAA_940.jpg',
 'imageAAA_960.jp

In [24]:
files =  os.listdir(MyPhotos)
for fileName in files:
  try:
        # Gọi hàm trích xuất đặc trưng
      vector = myDetect(MyPhotos+'/'+fileName)

        # Kiểm tra việc trích xuất đặc trưng thành công:
      if vector is not None:
          #Thêm vector đặc trưng vào trong từ điển với nhãn là tên của thư mục
        print('Trích xuất đặc trưng thành công')

  except:
      # Bỏ qua nếu gặp lỗi
    pass

Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xu

In [25]:
# Apply for all images
# Khởi tạo một dictionary
data = dict(data=[], label=[])

myDir = '/content/drive/MyDrive/opencv/FaceImage'
folders = os.listdir(myDir)

# Duyệt qua từng thư mục
for folder in folders:
  path = myDir + "/"+folder
  print(path)

  files =  os.listdir(path)
  for fileName in files:
    try:
        # Gọi hàm trích xuất đặc trưng
        vector = myDetect(path+'/'+fileName)

        # Kiểm tra việc trích xuất đặc trưng thành công:
        if vector is not None:
          #Thêm vector đặc trưng vào trong từ điển với nhãn là tên của thư mục
          data['data'].append(vector)
          data['label'].append(folder)
          print('Trích xuất đặc trưng thành công')
    except:
      # Bỏ qua nếu gặp lỗi
      pass

/content/drive/MyDrive/opencv/FaceImage/1721031693_HuynhMinhTien
/content/drive/MyDrive/opencv/FaceImage/1721031651_NguyenHoangKyAnh
/content/drive/MyDrive/opencv/FaceImage/1721031695_DinhDuyLong
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
/content/drive/MyDrive/opencv/FaceImage/1721031561_NguyenDucHieu
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trí

In [26]:
# Tạo một series từ dictionary
labelSeries = pd.Series(data['label'])
# Đếm số lần xuất hiện của mỗi nhãn
labelCounts = labelSeries.value_counts()
print(labelCounts)

1721030566_LeNguyenHoang         150
1721030848_LeQuocKy              135
1721031129_TranAnhHao            130
172100257_LuongThanhPhong        121
1721030367_LePhuongLinh          119
                                ... 
1721030545_NguyenHoangTuanAnh     10
1721030497_HoThienSinh             9
172100103_tranquangthuan           8
1721021119_TranDuyHa               6
1721030802_NguyenNhatTien          5
Name: count, Length: 93, dtype: int64


In [27]:
# Save data -> file
print('/content/drive/MyDrive/opencv/')
fileName = '/content/drive/MyDrive/opencv/data_face_features.pickle'
pickle.dump(data, open(fileName, mode='wb'))

/content/drive/MyDrive/opencv/
